In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
cd ~/demtext

/Users/zurfarosa/demtext


In [43]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from IPython.display import display

In [44]:
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', -1)
pd.options.display.html.use_mathjax = False #stops Jupyter from interpreting dollar signs in the data as markdown instructions

# Load entries and annotations

In [45]:
entries = pd.read_csv('all_entries.csv',usecols=['entry id','patient id','gender','birthyear','Dementia','Diagnostic category','Age_at_diagnosis','free text','Read code','Read term','eventdate'])

In [46]:
entries = entries.rename(columns={'patient id':'patid','Diagnostic category':'dx_category'})

In [47]:
entries=entries[pd.notnull(entries['patid'])]

In [48]:
#remove the one entry where there is no diagnostic category
entries = entries[entries['dx_category']!=" "]

In [49]:
entries.head(5)

entry id   patid  gender  birthyear  Age_at_diagnosis  Dementia  \
0  FN00001  380163  2       1916       91                0          
1  FN00002  380163  2       1916       91                0          
2  FN00003  380163  2       1916       91                0          
3  FN00004  380163  2       1916       91                0          
4  FN00005  380163  2       1916       91                0          

                                                                                               free text  \
0  NaN                                                                                                     
1  NaN                                                                                                     
2  NaN                                                                                                     
3  R EYE NOW LOWER LID INFLAMED AND WATERING ++. PLAN TREAT INF THEN REV ? WILL NEED REF TO OPTHALMOLOGY   
4  NaN                                                                                                     

  Read code                   Read term  eventdate dx_category  
0  442J.00   Thyroid function test       38908      FN          
1  442V.00   Serum free T4 level         38908      FN          
2  442W.00   Serum TSH level             38908      FN          
3  F4E1.00   Ectropion                   38960      FN          
4  1A26.00   Urge incontinence of urine  38965      FN

In [50]:
annotations = pd.read_csv("annotations.csv")

In [51]:
annotations = annotations.rename(columns={'patient id':'patid','Tag':'tag'})

In [52]:
annotations.sample(5)

entry id       patid                                     Read code  \
724  TP12312  4145538.0   #O/E - blood pressure reading                  
108  FN00428  1009360.0   #Referral to primary care mental health team   
379  FN03815  7327120.0   #Seen in psychiatry clinic                     
317  FN02449  4745491.0   #Home visit                                    
561  FP07558  12142360.0  #[X]Dementia in Alzheimer's disease            

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# Create pt_features

In [53]:
unique_patids = list(set(entries['patid']))

In [54]:
pt_features = pd.DataFrame({'patid':unique_patids})

In [55]:
pt_features = pd.merge(pt_features,entries,how='left')[['patid','gender','birthyear','Age_at_diagnosis','Dementia']].drop_duplicates()

In [56]:
pt_features[pt_features['patid']==27947181]

patid  gender  birthyear  Age_at_diagnosis  Dementia
5311  27947181  2       1923       83                0

In [57]:
len(pt_features)

255

In [58]:
expert_diagnoses = pd.read_csv('expert_diagnoses.csv')

In [59]:
pt_features = pd.merge(pt_features,expert_diagnoses,how='left',on='patid')

In [60]:
len(pt_features)

255

In [63]:
pt_features = pd.merge(pt_features,entries[['patid','dx_category']],how='left',on='patid').drop_duplicates()

In [64]:
pt_features

patid  gender  birthyear  Age_at_diagnosis  Dementia      expert_dx  \
0      5227009   2       1922       84                0         definite        
109    10536450  1       1923       77                1         NaN             
120    1883652   2       1925       78                1         NaN             
140    19589125  2       1920       88                0         NaN             
179    13942279  2       1924       83                0         NaN             
182    2180104   2       1916       91                0         definite        
305    5433355   2       1917       92                0         uncertain       
368    3773442   2       1914       91                0         NaN             
391    13536270  1       1927       83                0         probable        
480    25084431  2       1918       92                1         NaN             
587    30851088  1       1924       78                0         NaN             
600    894481    2       1916       90                0         probable        
686    4870162   2       1917       83                1         NaN             
711    10170115  2       1912       90                1         NaN             
730    14529506  1       1918       94                1         NaN             
877    10171441  2       1921       89                1         NaN             
968    4670586   1       1927       82                1         NaN             
1041   641564    2       1915       93                0         definite        
1118   13882398  2       1918       83                1         NaN             
1142   14242480  2       1919       84                1         NaN             
1193   565282    1       1913       92                0         NaN             
1254   31088164  2       1919       87                0         uncertain       
1318   4701493   2       1920       92                0         NaN             
1358   4544551   2       1913       91                1         NaN             
1434   2560553   1       1917       90                0         NaN             
1446   4539435   2       1917       90                1         NaN             
1529   20293164  2       1924       87                0         NaN             
1613   8275501   1       1924       85                0         probable        
1707   3176494   2       1931       78                0         uncertain       
1775   2830384   2       1916       92                0         NaN             
1776   9905201   2       1919       85                1         NaN             
1800   6214194   1       1925       82                1         NaN             
1949   8923187   1       1921       90                1         NaN             
2018   7107125   1       1916       93                1         NaN             
2106   20217399  2       1929       75                1         NaN             
2111   6548541   2       1928       78                0         probable        
2157   2220607   2       1919       87                1         NaN             
2247   4439617   1       1936       75                1         NaN             
2320   13883403  2       1924       85                1         NaN             
2390   12074567  1       1921       86                0         uncertain       
2439   11925064  2       1918       89                1         NaN             
2477   19126345  2       1917       92                1         NaN             
2498   4379212   1       1919       91                0         probable        
2580   12240466  1       1914       94                1         NaN             
2768   598611    2       1916       88                1         NaN             
2829   24272164  2       1914       89                1         NaN             
2842   15276125  1       1919       89                1         NaN             
2899   1098334   1       1912       88                0         NaN             
2933   4560479   2       1928       83   

# Perform tag counts

In [23]:
# Make sure that max count for a specific tag per entry is 1

In [65]:
annotation_tag_count = annotations.groupby(['entry id','tag']).patid.nunique().reset_index()[['entry id','tag']]

In [66]:
annotation_tag_count = pd.merge(annotation_tag_count,annotations[['entry id','patid']],how='left',on='entry id').sort_values(by='patid').drop_duplicates()

In [67]:
annotation_tag_count['tag'] = 'tag:' + annotation_tag_count['tag'].astype(str)


In [68]:
annotation_tag_count.head(5)

entry id                                              tag     patid
705  TP10979  tag:not coping / in need of more care            166256.0
686  TP10974  tag:family member closely involved in pt's care  166256.0
691  TP10974  tag:medication mistakes                          166256.0
694  TP10974  tag:memory problems                              166256.0
698  TP10977  tag:memory problems                              166256.0

In [69]:
annotation_tag_count = annotation_tag_count.groupby(['patid','tag'])

In [70]:
annotation_tag_count = annotation_tag_count.count().reset_index()


In [71]:
annotation_tag_count = annotation_tag_count.rename(columns={'entry id':'tag_count'})

In [72]:
annotation_tag_count[annotation_tag_count['patid']==894481]

patid                                              tag  tag_count
46  894481.0  tag:DNA'ing appointments                         2        
47  894481.0  tag:MMSE                                         1        
48  894481.0  tag:dementia medication                          4        
49  894481.0  tag:dementia screen                              1        
50  894481.0  tag:family member closely involved in pt's care  3        
51  894481.0  tag:medication mistakes                          1        
52  894481.0  tag:memory clinic or mental health services      1

In [73]:
annotation_tag_count = annotation_tag_count.pivot(index='tag',columns='patid').transpose().reset_index().drop(columns=['level_0'])

In [74]:
del annotation_tag_count.columns.name

In [75]:
annotation_tag_count

patid  tag:6-CIT  tag:ACE  tag:AMTS  tag:CAMCOG  \
0    166256.0   NaN        NaN       1.0      NaN           
1    340294.0   NaN        NaN      NaN       NaN           
2    380163.0   NaN        NaN      NaN       NaN           
3    612249.0   NaN        NaN      NaN       NaN           
4    641564.0   NaN        NaN      NaN       NaN           
5    777172.0   NaN        NaN      NaN       NaN           
6    839353.0   NaN        NaN      NaN       NaN           
7    894481.0   NaN        NaN      NaN       NaN           
8    902330.0   NaN        NaN      NaN       NaN           
9    1009360.0  NaN        NaN      NaN       NaN           
10   1153280.0  NaN        NaN      NaN       NaN           
11   1155452.0  NaN        NaN      NaN       NaN           
12   1158290.0  NaN        NaN      NaN       NaN           
13   1242325.0  NaN        NaN      NaN       NaN           
14   1267539.0  NaN        NaN      NaN       NaN           
15   1536668.0  NaN        NaN      NaN       NaN           
16   1895038.0  NaN        NaN      NaN       NaN           
17   1953522.0  NaN         1.0     NaN       NaN           
18   1983470.0  NaN        NaN      NaN       NaN           
19   2021011.0  NaN        NaN      NaN       NaN           
20   2085585.0  NaN        NaN      NaN       NaN           
21   2180104.0  NaN        NaN      NaN       NaN           
22   2303271.0  NaN        NaN      NaN       NaN           
23   2763624.0   2.0       NaN      NaN       NaN           
24   3149320.0  NaN        NaN      NaN       NaN           
25   3176494.0  NaN        NaN      NaN       NaN           
26   3665071.0  NaN        NaN      NaN       NaN           
27   3785435.0  NaN        NaN      NaN       NaN           
28   3872524.0  NaN        NaN      NaN       NaN           
29   3922090.0  NaN        NaN      NaN       NaN           
30   4052252.0  NaN        NaN      NaN       NaN           
31   4145538.0  NaN        NaN      NaN       NaN           
32   4262111.0  NaN        NaN      NaN       NaN           
33   4345152.0  NaN        NaN      NaN       NaN           
34   4379212.0  NaN        NaN      NaN       NaN           
35   4560479.0  NaN        NaN      NaN       NaN           
36   4701493.0  NaN        NaN      NaN       NaN           
37   4745491.0  NaN        NaN      NaN       NaN           
38   4758519.0  NaN         1.0     NaN       NaN           
39   4790541.0  NaN        NaN      NaN       NaN           
40   4990366.0  NaN        NaN      NaN       NaN           
41   5083164.0  NaN        NaN      NaN       NaN           
42   5134670.0  NaN        NaN      NaN       NaN           
43   5227009.0  NaN        NaN      NaN       NaN           
44   5433355.0  NaN        NaN      NaN       NaN           
45   5608079.0  NaN        NaN      NaN       NaN           
46   5622194.0  NaN        NaN      NaN       NaN           
47   5966165.0  NaN        NaN      NaN       NaN           
48   6214194.0  NaN        NaN      NaN       NaN           
49   6228456.0   1.0       NaN      NaN       NaN           
50   6451064.0  NaN        NaN      NaN       NaN           
51   6548541.0  NaN        NaN      NaN       NaN           
52   6975195.0  NaN        NaN      NaN       NaN           
53   7107125.0  NaN        NaN      NaN       NaN           
54   7116125.0  NaN        NaN      NaN       NaN           
55   7123647.0  NaN        NaN      NaN       NaN           
56   7327120.0  NaN        NaN      NaN       NaN           
57   7470411.0  NaN        NaN      NaN       NaN           
58   7719319.0  NaN        NaN      NaN       NaN           
59   7736125.0  NaN        NaN      NaN       NaN           
60   7775125.0  NaN         1.0     NaN       NaN           
61   8076244.0  NaN        NaN      NaN       NaN           
62   8229110.0  NaN        NaN      NaN       NaN           
63   8270053.0   1.0       NaN      NaN       NaN           
64   8291322.0  NaN        NaN      NaN      

In [76]:
pt_features = pd.merge(pt_features,annotation_tag_count,how='left',on='patid')

In [77]:
pt_features.reset_index(inplace=True)

In [78]:
pt_features.drop('index',axis=1,inplace=True)

In [79]:
pt_features

patid  gender  birthyear  Age_at_diagnosis  Dementia      expert_dx  \
0    5227009   2       1922       84                0         definite        
1    10536450  1       1923       77                1         NaN             
2    1883652   2       1925       78                1         NaN             
3    19589125  2       1920       88                0         NaN             
4    13942279  2       1924       83                0         NaN             
5    2180104   2       1916       91                0         definite        
6    5433355   2       1917       92                0         uncertain       
7    3773442   2       1914       91                0         NaN             
8    13536270  1       1927       83                0         probable        
9    25084431  2       1918       92                1         NaN             
10   30851088  1       1924       78                0         NaN             
11   894481    2       1916       90                0         probable        
12   4870162   2       1917       83                1         NaN             
13   10170115  2       1912       90                1         NaN             
14   14529506  1       1918       94                1         NaN             
15   10171441  2       1921       89                1         NaN             
16   4670586   1       1927       82                1         NaN             
17   641564    2       1915       93                0         definite        
18   13882398  2       1918       83                1         NaN             
19   14242480  2       1919       84                1         NaN             
20   565282    1       1913       92                0         NaN             
21   31088164  2       1919       87                0         uncertain       
22   4701493   2       1920       92                0         NaN             
23   4544551   2       1913       91                1         NaN             
24   2560553   1       1917       90                0         NaN             
25   4539435   2       1917       90                1         NaN             
26   20293164  2       1924       87                0         NaN             
27   8275501   1       1924       85                0         probable        
28   3176494   2       1931       78                0         uncertain       
29   2830384   2       1916       92                0         NaN             
30   9905201   2       1919       85                1         NaN             
31   6214194   1       1925       82                1         NaN             
32   8923187   1       1921       90                1         NaN             
33   7107125   1       1916       93                1         NaN             
34   20217399  2       1929       75                1         NaN             
35   6548541   2       1928       78                0         probable        
36   2220607   2       1919       87                1         NaN             
37   4439617   1       1936       75                1         NaN             
38   13883403  2       1924       85                1         NaN             
39   12074567  1       1921       86                0         uncertain       
40   11925064  2       1918       89                1         NaN             
41   19126345  2       1917       92                1         NaN             
42   4379212   1       1919       91                0         probable        
43   12240466  1       1914       94                1         NaN             
44   598611    2       1916       88                1         NaN             
45   24272164  2       1914       89                1         NaN             
46   15276125  1       1919       89                1         NaN             
47   1098334   1       1912       88                0         NaN             
48   4560479   2       1928       83                0         uncertain       
49   1365600   2       1917       89                1     